![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
#region imports
from AlgorithmImports import *
#endregion
import pandas as pd
from footprint_consolidator import FootprintConsolidator
from datetime import datetime, timedelta

# --- 1. 初始化和历史数据请求 ---
qb = QuantBook()
symbol = qb.add_future(Futures.Metals.GOLD, Resolution.SECOND).symbol


# 请求足够的数据以形成一个完整的10分钟Bar
history_date = datetime(2024, 12, 5)
df_history = qb.history(symbol, history_date, history_date + timedelta(days=1),
                        resolution=Resolution.SECOND,
                        extended_market_hours=True,
                        data_mapping_mode=DataMappingMode.OPEN_INTEREST_ANNUAL, # 数据映射模式，这个会根据交易量切换到当年后续更大的合约，Warning, 但正确性有待验证
                        data_normalization_mode=DataNormalizationMode.BACKWARDS_RATIO, # 数据连续模式，ATAS是RAW, tradingview 是BACKWARDS_RATIO，能够使得连续。注意，实盘需要使用当期合约数据
                        fill_forward=True
                        )
print(df_history[df_history['volume']>=1])

if df_history.empty:
    qb.Debug("No history data found for the selected date or symbol.")
else:
    # --- 2. 生成一个10分钟的FootprintBar ---
    target_period = timedelta(minutes=10)
    sec = qb.Securities[symbol]
    tick_size = sec.SymbolProperties.MinimumPriceVariation
    
    # 使用我们的静态方法生成FootprintBar列表
    footprint_bars = FootprintConsolidator.create_from_history(df_history, target_period, tick_size)

    # --- 3. 检查第一个10分钟Bar的内容 ---
    if footprint_bars:
        first_bar = footprint_bars[0]
        
        print(f"--- 检查单个10分钟FootprintBar ---")
        print(f"时间范围: {first_bar.time} to {first_bar.end_time}")
        print(f"OHLC: {first_bar.open:.2f} / {first_bar.high:.2f} / {first_bar.low:.2f} / {first_bar.close:.2f}")
        print(f"总成交量: {first_bar.total_volume:.0f}")
        print(f"买入量: {first_bar.buy_volume:.0f}")
        print(f"卖出量: {first_bar.sell_volume:.0f}")
        print(f"Delta: {first_bar.delta:.0f}")
        print("-" * 50)
        print("各价位的成交量详情:")
        
        # 对价格进行排序以便查看
        sorted_prices = sorted(first_bar.volume_at_price.keys())
        
        if not sorted_prices:
            print("  该时间段内没有任何价位记录到成交量。")
        else:
            for price in sorted_prices:
                volumes = first_bar.volume_at_price[price]
                print(f"  价格: {price:8.2f} -> 买入量: {volumes['ask']:>8.0f} | 卖出量: {volumes['bid']:>8.0f}")
        
        print("-" * 50)
        
    else:
        print("未能生成FootprintBar。可能是输入数据不足。")
